In [0]:
#print  numbers
n=1
for i in range(10):
    print(i)
    n+=1
print(n)

In [0]:
%sql
select "I'm running sql"

In [0]:
%sql
select * from samples.nyctaxi.trips

In [0]:
display(dbutils.fs.ls("/databricks-datasets/"))

#built-in sample datasets that you can use for practice and testing

In [0]:
#display(dbutils.fs.ls("dbfs:/databricks-datasets/online_retail/"))#table1
#display(dbutils.fs.ls("dbfs:/databricks-datasets/retail-org/"))
display(dbutils.fs.ls("dbfs:/databricks-datasets/retail-org/products/"))





In [0]:
#display(dbutils.fs.ls("dbfs:/databricks-datasets/online_retail/data-001/"))#table 1 location
#display(dbutils.fs.ls("dbfs:/databricks-datasets/retail-org/products/products.csv")) #table2 location
display(dbutils.fs.ls("dbfs:/databricks-datasets/nyctaxi-with-zipcodes/subsampled/"))
display(dbutils.fs.ls("dbfs:/databricks-datasets/nyctaxi-with-zipcodes/"))


#dbutils = Databricks Utilities

In [0]:
df1 = spark.read.option("header", True).csv("dbfs:/databricks-datasets/online_retail/data-001/data.csv")


In [0]:
display(df1)

In [0]:
df1.printSchema
#it is treating all coulmns as string by providing proper schema  it will the columns as integer which columns having actual integer values

In [0]:
#for creating proper schema have created anaother dataframe
df2=spark.read.csv("dbfs:/databricks-datasets/online_retail/data-001/data.csv",header= True, inferSchema=True) 
df3 = spark.read.csv("dbfs:/databricks-datasets/retail-org/products/products.csv",header= True, inferSchema=True)
#here type has changed with respect to column type

In [0]:
df1.select("InvoiceNo","Quantity").show()

In [0]:
invoice_quantity = df1.select("InvoiceNo", "Quantity").collect()
spark.conf.set("spark.scheduler.pool", "high-priority") # it sets the priority of the job


In [0]:
df1.select(df1.columns[1:5]).collect()
#df1.select(df1.columns[1:5]).show()

In [0]:
dbutils.fs.put("/FileStore/tables/my_file.csv", open(r"C:\Users\harshitha\Desktop\Hospitals.csv",'r').read(), True)


In [0]:
df1.withColumn("UnitPrice1",df1["UnitPrice"]*100).show()

In [0]:
from pyspark.sql.functions import lit
df1.withColumn("place",lit("Notinghm")).show()

In [0]:
df1.printSchema()

In [0]:
from pyspark.sql.functions import col
df = df1.withColumn("InvoiceNo",df1["InvoiceNo"].cast("int"))

In [0]:
df.printSchema()

In [0]:
df.show()

In [0]:
df.withColumnRenamed("Country","Location").show()

In [0]:
#df.filter(df["UnitPrice"] > 10).show() #similar to where() in sql
#df.filter(df["UnitPrice"] > 10).select(df.columns[1:5]).show()
#df.filter((df["UnitPrice"] < 10) & (df["InvoiceNo"] == 536365)).show()
#df.filter((df["UnitPrice"] < 10) | (df["InvoiceNo"] == 536365)).show()
#df.filter((df["UnitPrice"] < 10)  (df["InvoiceNo"] == 536365)).show()
#df1.filter(df1["Country"]=="France").show()
df1.filter(df1.Country.startswith("F")).show()
df1.filter(df1.Country.contains("F")).show()
df1.filter(df1.Country.endswith("a")).show()
df1.filter(df1.Country.isin(["France","Spain"])).show()
df.filter(df.Country.eqNullSafe("France")).show()
df1.filter(df1.Country.like("%nia")).show()

In [0]:
df3=df1.distinct()
display(df3) #to display all records in the dataset
df3.dropDuplicates().show()
df3.dropDuplicates(["InvoiceNo"]).show()

In [0]:
#df1.sort(df1["InvoiceNo"]).display()
df1.sort(df1["InvoiceNo"],ascending=False).display() #means displays in descending order
#df1.sort(df1["InvoiceNo"],df1["UnitPrice"]).display()

In [0]:
#df1.groupBy("Country").count().display()

df1.groupBy("Country").agg({"UnitPrice":"sum"}).display()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()
# Create Orders DataFrame
orders_data = [
    (1, 101, 250.0, "2023-01-15"),
    (2, 102, 450.0, "2023-01-17"),
    (3, 101, 150.0, "2023-02-01"),
    (4, 103, 300.0, "2023-02-20")
]
orders_schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("amount", FloatType(), True),
    StructField("order_date", StringType(), True)
])
orders_df = spark.createDataFrame(data=orders_data, schema=orders_schema)
# Create Customers DataFrame
customers_data = [
    (101, "Alice", "USA", "USA"),
    (102, "Bob", "Canada", "CAN"),
    (103, "Charlie", "UK", "UK"),
    (104, "David", "Australia", "AUS")
]
customers_schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("customer_name", StringType(), True),
    StructField("country", StringType(), True),
    StructField("country_shortform", StringType(), True) #to balanced the number of columns
])
customers_df = spark.createDataFrame(data=customers_data, schema=customers_schema)

# Show the DataFrames
orders_df.show()
customers_df.show()



In [0]:
# Union operation is not possible between orders_df and customers_df due to schema mismatch.
# Instead, we can join the DataFrames on customer_id.

union_df = orders_df.join(customers_df, on="customer_id", how="inner")
display(union_df)

In [0]:
#orders_df.union(customers_df).display()
orders_df.unionAll(customers_df).display()#not recommended
orders_df.union(customers_df).distinct().display() #if you want unique values

In [0]:
df1.fillna("unkonown").display()
df1.fillna(0,subset=["InvoiceNo"]).fillna("unkonown",["Country"]).display()

In [0]:
orders_df.printSchema()

In [0]:
orders_df.write.format("delta").mode("overwrite").saveAsTable("default.orders_table")
customers_df.write.format("delta").mode("overwrite").saveAsTable("default.customers_table")


In [0]:
%sql
SHOW DATABASES

In [0]:
%sql
CREATE DATABASE Harshithadb

In [0]:
%sql
--DESCRIBE  samples.nyctaxi.trips
DESCRIBE EXTENDED samples.nyctaxi.trips

